Copyright 2024 Lehi Gracia

### Save?

In [2]:
save_to_mongo = False

import subprocess, os
if save_to_mongo:
    mongod = subprocess.Popen(
        ['mongod', '--dbpath', os.path.expanduser(os.getenv('MONGODB_PATH')), '--logpath', os.path.expanduser(os.getenv('MONGODB_LOG'))]
    )

### MARKETS

In [3]:

import warnings
warnings.filterwarnings('ignore')

from yfinance_utils import list_utils, file_utils, constants
import yfinance 
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from finta import TA



COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
symbols = list_utils.get_markets()
names = list_utils.markets
    
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="MARKETS", title_x=0.5, width=600, height=400)

gains = {}
for symbol in symbols:
    data = file_utils.read_historic_data(symbol)
    data['sma'] = TA.SMA(data, period=50)
    data = data[50:]
    data.reset_index(inplace=True)

    fig.add_trace(
        go.Scatter(y=data['Close'], x=data['Date'], name=names[symbol]),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(y=data['sma'], x=data['Date'], name=f'sma'),
        secondary_y=False
    )

    temp_1 = data['Close'].iloc[-1]
    temp_2 = data['Close'].iloc[-30]
    gains[names[symbol]] = round(((temp_1 - temp_2) / temp_2 * 100),2)
fig.show()

print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol in symbols:
    print(f'{symbol:4}:{gains[names[symbol]]:6}: {names[symbol]}')



PERFORMANCE IN THE LAST 30 DAYS:
SPY :  2.97: S&P 500
DIA :  2.68: DOW JONES
QQQ :  2.23: NASDAQ 100


### Implied Volaility

In [4]:
import os
import yfinance
from finta import TA
from plotly.subplots import make_subplots



COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
_d = file_utils.read_historic_data('^VIX')
_d['sma'] = TA.SMA(_d, period=30)

_f=make_subplots(specs=[[{"secondary_y": True}]])
_f.update_layout(title=f"Market Volatility (VIX)", title_x=0.5, width=600, height=400)
_d.reset_index(inplace=True)
_f.add_trace(
    go.Scatter(y=_d['Close'], x=_d['Date'], name="VIX"),
    secondary_y=False
)
_f.add_trace(
    go.Scatter(y=_d['sma'], x=_d['Date'], name=f'sma'),
    secondary_y=False
)
_f.show()

### Sectors

In [5]:

from yfinance_utils import list_utils
from datetime import datetime

symbols = list_utils.get_sectors()
COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
TOP_OF_SECTOR = 3

fig2=make_subplots(rows=3, cols=1)
fig2.update_layout(title=f"TOP {TOP_OF_SECTOR} SECTORS", title_x=0.5, width=800, height=600)

top = list_utils.get_top_performers(symbols, size=TOP_OF_SECTOR)
_list = [i[0] for i in top]

counter=1
for symbol in _list:
    _d2 = pd.DataFrame()
    _d2 = file_utils.read_historic_data(symbol)
    _d2.columns = COLUMNS
    _d2['sma'] = TA.SMA(_d2, period=50)
    _d2.reset_index(inplace=True)
    fig2.add_trace(
        go.Scatter(y=_d2['Close'], x=_d2['Date'], name=symbol),
        secondary_y=False,
        row=counter, col=1
    )
    fig2.add_trace(
        go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma'),
        secondary_y=False,
        row=counter, col=1
    )
    counter = counter + 1
fig2.show()

today = datetime.today().strftime('%Y-%m-%d')
print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol, value in top:
    if save_to_mongo:
        file_utils.save_to_mongo({"DATE":today, "TICK":symbol, "PERFORMANCE":value, "SECTOR":symbol}, "top_sectors")



{'XLY': np.float64(5.03), 'XLV': np.float64(3.51), 'XLC': np.float64(4.22), 'XLI': np.float64(0.83), 'XLB': np.float64(1.54), 'XLK': np.float64(1.6), 'XLE': np.float64(4.56), 'XLU': np.float64(0.74), 'XLP': np.float64(-0.64), 'XAR': np.float64(-1.02), 'XLF': np.float64(2.53), 'XLRE': np.float64(0.26)}


PERFORMANCE IN THE LAST 30 DAYS:


###  Getting 5 best performing stocks in each of top 3 sectors

In [14]:
# Best performance stock in sectors

from yfinance_utils import list_utils
import numpy as np

TOP_OF_SECTOR = 5
SMA_PERIOD = 50
_sym = {}
for symbol, value in top:
    _l = []
    for s in list_utils.SECTORS[symbol].keys():
        _l.append(s)
    _tp = list_utils.get_top_performers(_l, days=30, size=TOP_OF_SECTOR)
    _l = [i[0] for i in _tp]
    _sym[symbol] = _l
    
for i in list(_sym.keys()):
    
    _f=make_subplots(rows=TOP_OF_SECTOR, cols=1)
    _f.update_layout(title=f"TOP {TOP_OF_SECTOR} from {i}", title_x=0.5, width=600, height=1000)
    
    counter = 1
    for symbol in _sym[i]:
        _d2 = pd.DataFrame()
        _d2 = file_utils.read_historic_data(symbol)
        _d2.columns = COLUMNS
        _d2['sma'] = TA.SMA(_d2, period=SMA_PERIOD)
        _d2['pct'] = _d2['sma'].pct_change()
        _d2['pct_close'] = _d2['Close'] * _d2['pct'] * 25

        _d2["Color"] = np.where(_d2["pct_close"] < 0, 'red', 'green')

        _d2.reset_index(inplace=True)
        _f.add_trace(
            go.Scatter(y=_d2['Close'], x=_d2['Date'], name=symbol,showlegend=False),
            col=1, row=counter
        )
        _f.add_trace(
            go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma {SMA_PERIOD}', showlegend=False),
            col=1, row=counter
        )
        _f.add_trace(
            go.Bar(y=_d2['pct_close'], x=_d2['Date'], name=f'change', marker_color=_d2['Color'], showlegend=False),
            col=1, row=counter
        )
        _f.update_yaxes(side='right')
        price = f"{symbol} - Date: {today}, Open:{round(_d2['Open'].iloc[-1],2)}, High:{round(_d2['High'].iloc[-1],2)}, Low:{round(_d2['Low'].iloc[-1],2)}, Close: {round(_d2['Close'].iloc[-1],2)}"
        _f.add_annotation(xref='x domain',
                        yref='y domain',
                        x=0.01,
                        y=1,
                        text=price, 
                        showarrow=False,
                        font=dict(size=10),
                        row=counter, col=1)
        counter = counter + 1
    _f.show()

if save_to_mongo:
    _sym['DATE'] = datetime.today().strftime('%Y-%m-%d')
    file_utils.save_to_mongo(_sym, "top_performers")

{'AMZN': np.float64(2.43), 'TSLA': np.float64(3.84), 'HD': np.float64(13.03), 'MCD': np.float64(4.74), 'BKNG': np.float64(-0.54), 'LOW': np.float64(17.15), 'TJX': np.float64(12.07), 'SBUX': np.float64(-6.68), 'NKE': np.float64(6.61), 'CMG': np.float64(-21.91), 'ORLY': np.float64(12.93), 'MAR': np.float64(-1.59), 'HLT': np.float64(0.2), 'GM': np.float64(9.62), 'ABNB': np.float64(-6.68), 'RCL': np.float64(4.9), 'AZO': np.float64(13.45), 'ROST': np.float64(14.38), 'LULU': np.float64(-11.02), 'DHI': np.float64(27.61), 'F': np.float64(4.26), 'YUM': np.float64(-3.07), 'GRMN': np.float64(5.88), 'LEN': np.float64(20.15), 'DECK': np.float64(16.42), 'EBAY': np.float64(17.99), 'TSCO': np.float64(10.2), 'CCL': np.float64(8.65), 'NVR': np.float64(8.1), 'EXPE': np.float64(16.1), 'PHM': np.float64(20.05), 'DRI': np.float64(-1.3), 'ULTA': np.float64(6.4), 'LVS': np.float64(17.56), 'BBY': np.float64(7.39), 'GPC': np.float64(13.55), 'TPR': np.float64(-1.48), 'APTV': np.float64(16.88), 'DPZ': np.float64(

### Top Analysts Ratings

In [12]:

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
pd.set_option('display.max_colwidth', None)

client = MongoClient('mongodb://localhost:27017/')
db = client['bigdata']
collection = db.market

yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
today = datetime.today().strftime('%Y-%m-%d')


res = collection.find(
                 {'DATE':yesterday, 'script':'daily_analysts_ratings'}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2).sort_values(by="TOTAL", axis=0, ascending=False).drop_duplicates(subset=["TICK"], keep='last').iloc[:20]
if data.empty:
    print('No Data Today')
else:
    data.set_index('DATE', inplace=True)
    print(data)


KeyboardInterrupt: 

### Gap up close high

In [13]:
# Gap up close high

res = collection.find(
                 {'DATE':yesterday, 'script':{'$in':['daily_gap_up_close_high']}}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
if data.empty:
    print('No Data Today')
else:
    data.set_index('DATE', inplace=True)
    print(data)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 68b1e581a2dcdfcd13eef680, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

### Price drops and jumps

In [ ]:
# Price Ups and Downs

res = collection.find(
                 {'DATE':yesterday, 'script':{'$in':['daily_jump_price_down','daily_jump_price_up']}}, 
                 {'_id':0, 'script':0, 'timestamp':0, 'VOLUME':0  }
                 )

data = pd.DataFrame(res).round(2)
if data.empty:
    print('No Data Today')
else:
    data.set_index('DATE', inplace=True)
    print(data)

            TICK   CLOSE  % PRICE JUMP  % VOLUME -1  % VOLUME -2
DATE                                                            
2025-01-21  MPWR  637.09          5.04        13.57       115.42
2025-01-21  XPEV   14.92          6.72        69.29       -28.38
2025-01-21   SMR   23.86         13.62        30.64        76.93
2025-01-21  MRNA   35.80          5.79        57.36         8.30
2025-01-21   VST  179.30          6.08        24.77        43.01
2025-01-21   VVX   54.79          5.06       177.45       -44.10
2025-01-21   NNE   26.63         11.10       158.76         6.99
2025-01-21  QRVO   87.15         11.73       -51.19       505.75
2025-01-21  OKLO   27.62         10.52        69.54        32.66
2025-01-21  CDRE   38.76          6.63        76.29         4.33
2025-01-21   PDD  108.07          6.47       -27.22       113.33
2025-01-21  QBTS    5.40         -5.76        70.16       -53.43
2025-01-21   WBA   11.72        -10.05        48.86        81.20
2025-01-21   APA   23.99 

### TTM Squeeze

In [ ]:
# TTM SQUEEZE

res = collection.find(
    {"script":"daily_ttm_squeeze", "DATE":yesterday},
    {"_id":0, "script":0, "timestamp":0}
    )

df = pd.DataFrame(res)
df.set_index("DATE", inplace=True)
print(df)

            TICK  DAYS
DATE                  
2025-01-21   CAH    10
2025-01-21   MMM    46
2025-01-21  XPEV    27
2025-01-21   HWM     9
2025-01-21   VLO     7
...          ...   ...
2025-01-21  ISRG    32
2025-01-21   TPR     6
2025-01-21    ET    22
2025-01-21   DAL    10
2025-01-21   ETR    15

[72 rows x 2 columns]


In [ ]:

print('The END')

The END
